# imported libraries

In [54]:
from newspaper import Article
import numpy as np
import nltk
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from keras.regularizers import l1, l2, l1_l2
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend
from keras import optimizers
from keras import layers
from keras.callbacks import LearningRateScheduler
from tkinter import *

In [9]:
root=Tk()
root.geometry('500x500')
root.title('News Bias Classifier and Summarizer')
root.rowconfigure(0, weight=1)
root.columnconfigure(0, weight=1)
root.state('zoomed')

url_paste=Frame(root,height=500, width=500, cursor="arrow", bg='gray')
L1=Label(url_paste, text="Enter url", font="arial 15", fg="red")
L1.place(x=200, y=200)

def get_val():
    global url
    url = E1.get()
    root.destroy()

global E1 
E1=Entry(url_paste, width=40)
E1.place(x=300, y=200)

B1=Button(url_paste, text="find bias", command=lambda:get_val(), width=20, height=2)
B1.place(x=200, y=400)


url_paste.grid(row=0, column=0, sticky='nsew')
url_paste.columnconfigure(0, weight=1)
url_paste.rowconfigure(1, weight=1)
root.mainloop()

In [10]:
art = Article(url, 'en')
url

'https://edition.cnn.com/2022/12/10/politics/kyrsten-sinema-independent-democrats'

In [11]:
art.download()
art.parse()

In [12]:
art.authors

['Harry Enten']

In [13]:
art.publish_date

datetime.datetime(2022, 12, 10, 0, 0)

In [28]:
news=art.text
news

'CNN —\n\nArizona Sen. Kyrsten Sinema decided to shake up the political world on Friday by becoming an independent. The former Democrat is still caucusing with the party in the Senate, so the Democratic caucus still has 51 members. Now, instead of 49 Democrats and two independents within their ranks, the caucus has 48 Democrats and three independents.\n\nBut that simple math hides a more clouded picture for Democrats and for Sinema herself. Sinema’s interests are no longer necessarily the Democrats’ best interests in the next Congress, and the 2024 Senate map became even more complicated for Democrats with Sinema’s decision.\n\nTo be clear, Sinema has always been a thorn in the Democrats side during her time in Congress. Over the last two years, Democrats have had to almost always make sure that any bill or nomination had Sinema’s support to have any chance of passing. That’s the math when you have only 50 Senate seats in a 100-seat chamber. A lot of bills and nominations were never vo

In [15]:
from sentence_transformers import SentenceTransformer

In [17]:
s_model = SentenceTransformer(r"C:\Users\akkud\Machine Learning Project\bert_model")

In [31]:
news_data=s_model.encode(news)

# embed the news article using universal encoder

In [33]:
print(news_data.shape)
news_data=np.reshape(news_data, (1,-1))

(768,)


In [34]:
news_data

array([[-6.04453444e-01,  2.36399323e-01, -6.64811313e-01,
        -7.95824081e-02,  1.26201570e-01, -8.30531955e-01,
         4.19678479e-01, -6.02998674e-01, -2.76860297e-01,
        -3.36546302e-01,  4.33268368e-01,  2.41024539e-01,
         2.41031989e-01, -3.89277898e-02, -1.28261077e+00,
         2.38878220e-01,  2.24714965e-01, -5.87052479e-02,
        -4.83524054e-02, -7.74259120e-02,  6.38847053e-02,
        -8.15779269e-02,  2.37195473e-02,  4.63554680e-01,
         7.53658891e-01,  5.35550714e-01, -3.11892301e-01,
        -6.59392029e-02, -6.83207333e-01,  2.31067151e-01,
        -5.64667881e-01, -3.83372307e-02, -4.90362078e-01,
         1.52713358e-01,  6.22927129e-01,  7.02196062e-01,
         1.89165175e-01, -3.49001378e-01,  2.79880673e-01,
         3.88891041e-01, -2.09539473e-01, -9.34853405e-02,
        -2.19796211e-01, -1.11207768e-01, -7.22462177e-01,
        -1.54075786e-01,  4.56569314e-01,  1.07360756e+00,
         1.00872242e+00, -1.14205599e+00,  4.19094324e-0

In [35]:
from keras.models import load_model

In [36]:
news_model = load_model(r'C:\Users\akkud\Machine Learning Project\DNN_model')

In [37]:
ans = np.argmax(news_model.predict(news_data))
ans

1/1 [==============================] - 0s 46ms/step


1

In [38]:
if ans==0:
    x='News Article is biased towards left side'
    
if ans==1:
    x='News Article is biased towards right side'

In [39]:
#functions to preprocess the article
def casefolding(sentence):
    return sentence.lower()

def cleaning(sentence):
    return re.sub(r'[^a-z]', ' ', re.sub("’", '', sentence))

def tokenization(sentence):
    return sentence.split()

def sentence_split(paragraph):
    return nltk.sent_tokenize(paragraph)

def word_freq(data):
    w = []
    for sentence in data:
        for words in sentence:
            w.append(words)
    bag = list(set(w))
    res = {}
    for word in bag:
        res[word] = w.count(word)
    return res

def sentence_weight(data):
    weights = []
    for words in data:
        temp = 0
        for word in words:
            temp += wordfreq[word]
        weights.append(temp)
    return weights

In [40]:
#create list of sentences from the given article using the function
sentence_list = sentence_split(news)
data = []
for sentence in sentence_list:
    data.append(tokenization(cleaning(casefolding(sentence))))
data = (list(filter(None, data)))

In [41]:
wordfreq = word_freq(data)
rank = sentence_weight(data)


In [42]:
n = 4
result = ''
sort_list = np.argsort(rank)[::-1][:n]
for i in range(n):
    result += '{} '.format(sentence_list[sort_list[i]])

In [43]:
result = result.replace('Advertisement', '')
result = result.replace('\n', '')
result

'The idea being that Sinema still caucuses with the Democrats, and Democrats wouldn’t want to split the Democratic vote in a general election allowing a Republican to win in a purple state like Arizona. Ned Lamont)Relative to the average Senate Democrat, Lieberman voted with the party 10 points less of the time after becoming an independent than he had in his last term as a Democrat. Having two people in the race who are going to caucus with the Democratic Party likely makes it more difficult for the Democrats to win. The longtime Democrat won reelection as a third-party candidate in 2006, after losing the Democratic primary to a left-wing challenger (the now fairly moderate Connecticut Gov. '

In [52]:
root=Tk()
root.geometry('500x500')
root.title('News Bias Classifier and Summarizer')
root.rowconfigure(0, weight=1)
root.columnconfigure(0, weight=1)
root.state('zoomed')
T1 = Text(root, height = 500, width = 200, font = 'arial 15', padx=10, pady=10)
L2 = Label(root, text=x, font = 'arial 18')
b = Button(root, text = "Exit", command = lambda:root.destroy(), width=20, height=2, font='arial 15')

T1.pack()
b.place(x=600, y=500)
L2.place(x=550, y=300)
T1.insert(END, result)
root.mainloop()

In [55]:
xx=np.load('embedded_data.npy')

MemoryError: Unable to allocate 164. MiB for an array with shape (42896640,) and data type float32